## plot time series of hindcast and forecast at lead year ii

In [1]:
import matplotlib.pyplot as plt
import xarray as xr
import netCDF4
import numpy as np
from scipy.stats import pearsonr
import os

In [2]:
USER = os.environ['USER']

## FOSI hindcast 

In [3]:
datadir = f'/glade/scratch/{USER}/DPLE-results'
filename = f'{datadir}/DPLE-FOSI_hindcast_Phi_Eo_space_LME_1948-2017_yearly.nc'
ds_hind = xr.open_dataset(filename)
ds_hind

<xarray.Dataset>
Dimensions:          (lme: 11, year: 70)
Coordinates:
  * lme              (lme) int64 1 2 3 4 5 6 7 8 9 10 65
    time             (year) int64 ...
Dimensions without coordinates: year
Data variables: (12/26)
    Phi_0_0-200m     (lme, year) float64 ...
    Phi_0_200-600m   (lme, year) float64 ...
    Phi_1_0-200m     (lme, year) float64 ...
    Phi_1_200-600m   (lme, year) float64 ...
    Phi_2_0-200m     (lme, year) float64 ...
    Phi_2_200-600m   (lme, year) float64 ...
    ...               ...
    Phi_10_0-200m    (lme, year) float64 ...
    Phi_10_200-600m  (lme, year) float64 ...
    Phi_11_0-200m    (lme, year) float64 ...
    Phi_11_200-600m  (lme, year) float64 ...
    Phi_12_0-200m    (lme, year) float64 ...
    Phi_12_200-600m  (lme, year) float64 ...

## DPLE forecast

In [29]:
#varr = 'O2'
#varr = 'AOU'
#varr = 'TEMP'
varr = 'SALT'

In [30]:
datadir = f'/glade/scratch/{USER}/DPLE-results'
filename = f'{datadir}/DPLE_driftcorrected_Phi_components_{varr}_LME_ens_mean_yearly.nc'
ds_fore = xr.open_dataset(filename)
ds_fore

<xarray.Dataset>
Dimensions:               (lme: 11, year: 10, Y: 64)
Coordinates:
  * lme                   (lme) int64 1 2 3 4 5 6 7 8 9 10 65
  * year                  (year) int64 0 1 2 3 4 5 6 7 8 9
  * Y                     (Y) int64 1954 1955 1956 1957 ... 2014 2015 2016 2017
Data variables: (12/26)
    Phi_SALT_0_0-200m     (lme, year, Y) float64 ...
    Phi_SALT_0_200-600m   (lme, year, Y) float64 ...
    Phi_SALT_1_0-200m     (lme, year, Y) float64 ...
    Phi_SALT_1_200-600m   (lme, year, Y) float64 ...
    Phi_SALT_2_0-200m     (lme, year, Y) float64 ...
    Phi_SALT_2_200-600m   (lme, year, Y) float64 ...
    ...                    ...
    Phi_SALT_10_0-200m    (lme, year, Y) float64 ...
    Phi_SALT_10_200-600m  (lme, year, Y) float64 ...
    Phi_SALT_11_0-200m    (lme, year, Y) float64 ...
    Phi_SALT_11_200-600m  (lme, year, Y) float64 ...
    Phi_SALT_12_0-200m    (lme, year, Y) float64 ...
    Phi_SALT_12_200-600m  (lme, year, Y) float64 ...

## Calculate Corr/Rcri Matrix for Lead Year v. LME box#

In [36]:
var = 'Phi'
layer = '0-200m'
#layer = '200-600m'

In [37]:
import tools
alpha = 0.05
# time period for evaluation 1954-2007
yr1 = 1954
yr2 = 2007
# lead year range: 1,2,...,9,10
ldyrs = range(1, 11)
# persistence ACC: var1; var2
# DPLE ACC:        var2; var3
lme = [1,2,3,4,5,6,7,8,9,10,65]

lmen = ['EBS','GoA','CC','GoC','GoM','SEUS','NEUS','SS','LN','IPH','AI']
# 9/13 phis
#nphi = 9
nphi = 13
#
corr_p = np.ma.zeros([nphi, len(lme), len(ldyrs)]); corr_p.mask=True
rcri_p = np.ma.zeros([nphi, len(lme), len(ldyrs)]); rcri_p.mask=True
corr_d = corr_p.copy()
rcri_d = rcri_p.copy()
corr13 = corr_p.copy()
rcri13 = rcri_p.copy()
nmae_p = corr_p.copy()
nmae_d = corr_p.copy()
#
for cc in range(nphi):
    for nn in range(len(lme)):
        box = lme[nn]
        var1 = ds_hind[f'{var}_{cc}_{layer}'].sel(lme=box)[yr1-1948:yr2-1948+1].values
        #var1c = np.nanmean(ds_hind[f'{var}_{cc}_{layer}'].sel(lme=box)[1954-1948:2018-1948].values)
        #var1 = var1 - var1c
        #
        for lead in range(1, 11):
            var2 = ds_hind[f'{var}_{cc}_{layer}'].sel(lme=box)[yr1-1948+lead:yr2-1948+1+lead].values
            #var2c = np.nanmean()
            #var2 = var2 - var1c
            var3 = ds_fore[f'{var}_{varr}_{cc}_{layer}'].sel(lme=box)[lead-1, yr1-1954:yr2-1954+1].values
            #var3c = np.nanmean(ds_fore[f'{var}_{cc}_{layer}'].sel(lme=box)[1954-1954:2018-1954, lead-1].values)
            #var3 = var3 - var3c
            # mask nan values
            mask1 = np.isnan(var1)
            mask2 = np.isnan(var2)
            mask3 = np.isnan(var3)
            mem1 = np.ma.array(var1, mask=mask1)
            mem2 = np.ma.array(var2, mask=mask2)
            mem3 = np.ma.array(var3, mask=mask3)
            # Persistence ACC & NMAE
            fmask = np.logical_and(mem1.mask, mem2.mask)
            if np.sum(fmask) != len(fmask):
                nen1 = mem1[~fmask]
                nen2 = mem2[~fmask]
                corr_p[cc, nn, lead-1] = pearsonr(nen1, nen2)[0]
                neff = tools.neff3(nen1, nen2, len(nen1))
                rcri_p[cc, nn, lead-1] = tools.calculate_parson_corr_critical_value(neff, alpha)
                #
                #rmse_p[cc, nn, lead-1] = np.sqrt(np.nanmean((nen1-nen2)**2))
                nmae_p[cc, nn, lead-1] = np.nanmean(np.abs(nen1-nen2)/np.nanstd(nen2))
            # DPLE ACC & NMAE
            fmask = np.logical_and(mem2.mask, mem3.mask)
            if np.sum(fmask) != len(fmask):
                nen2 = mem2[~fmask]
                nen3 = mem3[~fmask]
                corr_d[cc, nn, lead-1] = pearsonr(nen2, nen3)[0]
                neff = tools.neff3(nen2, nen3, len(nen2))
                rcri_d[cc, nn, lead-1] = tools.calculate_parson_corr_critical_value(neff, alpha)
                #
                #rmse_d[cc, nn, lead-1] = np.sqrt(np.nanmean((nen2-nen3)**2))
                nmae_d[cc, nn, lead-1] = np.nanmean(np.abs(nen3-nen2)/np.nanstd(nen2))
            # Correlation for ACC Improvement Significance Test
            fmask = np.logical_and(mem1.mask, mem3.mask)
            if np.sum(fmask) != len(fmask):
                nen1 = mem1[~fmask]
                nen3 = mem3[~fmask]
                corr13[cc, nn, lead-1] = pearsonr(nen1, nen3)[0]
                neff = tools.neff3(nen1, nen3, len(nen1))
                rcri13[cc, nn, lead-1] = tools.calculate_parson_corr_critical_value(neff, alpha)
        #

In [38]:
data = xr.Dataset({"corr_p": (("nphi","nbox","nlead"),corr_p),\
                   "rcri_p": (("nphi","nbox","nlead"),rcri_p),\
                   "corr_d": (("nphi","nbox","nlead"),corr_d),\
                   "rcri_d": (("nphi","nbox","nlead"),rcri_d),\
                   "corr13": (("nphi","nbox","nlead"),corr13),\
                   "rcri13": (("nphi","nbox","nlead"),rcri13),\
                   "nmae_p": (("nphi","nbox","nlead"),nmae_p),\
                   "nmae_d": (("nphi","nbox","nlead"),nmae_d)})

In [39]:
dout = f'/glade/scratch/{USER}/DPLE-results'
data.load()

<xarray.Dataset>
Dimensions:  (nphi: 13, nbox: 11, nlead: 10)
Dimensions without coordinates: nphi, nbox, nlead
Data variables:
    corr_p   (nphi, nbox, nlead) float64 0.5867 0.2249 ... -0.05187 0.008762
    rcri_p   (nphi, nbox, nlead) float64 0.3882 0.3882 0.3961 ... 0.514 0.514
    corr_d   (nphi, nbox, nlead) float64 0.2433 0.3383 0.3114 ... 0.1148 0.04164
    rcri_d   (nphi, nbox, nlead) float64 0.4227 0.4227 0.4329 ... 0.576 0.5324
    corr13   (nphi, nbox, nlead) float64 0.1782 0.1969 0.212 ... -0.2081 -0.1665
    rcri13   (nphi, nbox, nlead) float64 0.4227 0.4227 0.4227 ... 0.5529 0.514
    nmae_p   (nphi, nbox, nlead) float64 0.7152 0.9936 1.122 ... 1.203 1.118
    nmae_d   (nphi, nbox, nlead) float64 23.56 23.3 23.33 ... 20.96 20.85 20.83

In [40]:
%%time
data.to_netcdf(f'{dout}/DPLE_LME_box01-11_ACC_NMAE_{var}_components_{varr}_Eo_space_{layer}_leadyear_1-10.nc', mode='w')

CPU times: user 9.92 ms, sys: 995 µs, total: 10.9 ms
Wall time: 12.6 ms
